In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv("Data_for_UCI_named.csv")

In [3]:
import matplotlib.pyplot as plt

In [5]:
df1.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [7]:
df1.stabf.value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [9]:

df1.isna().sum()


tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [38]:

#dropping stab column as its related with stabf
df=df1.drop('stab',axis=1)
df.columns


X_data=df.drop('stabf', axis=1)
Y_data=df[['stabf']]
     
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
y_train.value_counts()


     

unstable    5092
stable      2908
Name: stabf, dtype: int64

In [30]:


def scaler(df):
    scaler = MinMaxScaler()
    normalised_df = scaler.fit_transform(df)
    return pd.DataFrame(normalised_df, columns=df.columns)



In [31]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV



from sklearn.model_selection import train_test_split
x_train, x_test, y_train,y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=1)

In [32]:
def score_data(y_test,pred_y, clf):
  print('\n\n'+clf)
  print("\n CONFUSION MATRIX \n", confusion_matrix(y_test,pred_y))
  print(classification_report(y_test,pred_y))
  print("Accuracy Score is: ", accuracy_score(y_test, pred_y))
     

In [42]:

from sklearn.preprocessing import StandardScaler
myscaler=StandardScaler()
Xt_Scaled=pd.DataFrame(myscaler.fit_transform(x_train), columns=x_train.columns)
Xtest_Scaled=pd.DataFrame(myscaler.transform(x_test), columns=x_test.columns)

#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
Rf=RandomForestClassifier(n_estimators=100, random_state=1)
Rf.fit(Xt_Scaled,y_train)
pred_y=pd.Series(Rf.predict(Xtest_Scaled))
pred_y.value_counts()

unstable    1320
stable       680
dtype: int64

In [40]:


#Checking the accuracy, confusion matrix etc for Random Forest Classifier

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
score_data(y_test, pred_y,'RandomForest Classifier')
     




RandomForest Classifier

 CONFUSION MATRIX 
 [[ 625   87]
 [  55 1233]]
              precision    recall  f1-score   support

      stable       0.92      0.88      0.90       712
    unstable       0.93      0.96      0.95      1288

    accuracy                           0.93      2000
   macro avg       0.93      0.92      0.92      2000
weighted avg       0.93      0.93      0.93      2000

Accuracy Score is:  0.929


In [45]:
#LGBM Classifier

from lightgbm import LGBMClassifier
clf= LGBMClassifier()
clf.fit(Xt_Scaled,y_train)
pred_y=clf.predict(Xtest_Scaled)
score_data(y_test, pred_y,'LGBM')



LGBM

 CONFUSION MATRIX 
 [[ 641   71]
 [  50 1238]]
              precision    recall  f1-score   support

      stable       0.93      0.90      0.91       712
    unstable       0.95      0.96      0.95      1288

    accuracy                           0.94      2000
   macro avg       0.94      0.93      0.93      2000
weighted avg       0.94      0.94      0.94      2000

Accuracy Score is:  0.9395


In [46]:

n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None] 
hyperparameter_grid = {'n_estimators': n_estimators,'min_samples_leaf': min_samples_leaf,'min_samples_split': min_samples_split,'max_features': max_features}
from sklearn.model_selection import RandomizedSearchCV
etc_clf=ExtraTreesClassifier()
rdm = RandomizedSearchCV(etc_clf, param_distributions = hyperparameter_grid, cv=5, n_iter=10, n_jobs=-1, verbose=1, random_state=1, scoring='accuracy')
rdm.fit(Xt_Scaled, y_train)
rdm.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/toniadev/Downloads/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/toniadev/Downloads/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/toniadev/Downloads/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` o

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 8,
 'max_features': None}

In [47]:


etc_clf_opt=ExtraTreesClassifier(max_features=None,min_samples_leaf=8,min_samples_split=2, n_estimators=1000)
etc_clf_opt.fit(Xt_Scaled, y_train)
pred_y=etc_clf_opt.predict(Xtest_Scaled)
score_data(y_test, pred_y, clf='ExtraTreesClassifier Optimal')
     




ExtraTreesClassifier Optimal

 CONFUSION MATRIX 
 [[ 621   91]
 [  54 1234]]
              precision    recall  f1-score   support

      stable       0.92      0.87      0.90       712
    unstable       0.93      0.96      0.94      1288

    accuracy                           0.93      2000
   macro avg       0.93      0.92      0.92      2000
weighted avg       0.93      0.93      0.93      2000

Accuracy Score is:  0.9275


In [48]:
feature_imp=etc_clf_opt.feature_importances_

xyz=pd.DataFrame(feature_imp, index=Xt_Scaled.columns, columns=['Importance'])
print("Max Importance is of: ", xyz[['Importance']].idxmax())
print("Min Importance is of: ", xyz[['Importance']].idxmin())


Max Importance is of:  Importance    tau2
dtype: object
Min Importance is of:  Importance    p1
dtype: object
